In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

Lectura de .csv limpios

In [2]:
data = pd.read_csv("../../files/HR_RAW_DATA.csv", index_col = 0)

employees = pd.read_csv("../../files/df_exploracion_employees.csv" , index_col = 0)

experience = pd.read_csv("../../files/df_employee_exp.csv")

surveys = pd.read_csv("../../files/surveys.csv" , index_col = 0)

employment = pd.read_csv("transformacion_patri.csv" )

salary = pd.read_csv("../../files/salarydetails_w_floats.csv" , index_col = 0)

hours = pd.read_csv("../../files/df_Salary_SH.csv")

Concatenación de archivos

In [3]:
df_limpio = pd.concat([data['employeenumber'], employees, experience, surveys, employment, salary, hours], axis = 1, ignore_index = False)

Homogeneización de columnas

In [4]:
nuevas_columnas = {'employeenumber' : 'EmployeeID' , 'NUMBERCHILDREN' : 'NumberChildren' , 'NUMCOMPANIESWORKED' : 'NumCompaniesWorked', 'TOTALWORKINGYEARS' : 'TotalWorkingYears' , 'YEARSWITHCURRMANAGER' : 'YearsWithCurrManager' , 'WORKLIFEBALANCE' : 'WorkLifeBalance'}

In [5]:
df_limpio.rename(columns = nuevas_columnas, inplace = True)

Limpieza de duplicados

In [6]:
df_limpio = df_limpio.drop_duplicates(keep="first")

In [7]:
df_limpio = df_limpio.reset_index()
df_limpio.drop("index", axis=1, inplace=True)

In [8]:
df_limpio.head()

,EmployeeID,Age,Gender,MaritalStatus,NumberChildren,DateBirth,Over18,Education,EducationField,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Attrition,OverTime,DistanceFromHome,RemoteWork,BusinessTravel,TrainingTimesLastYear,Department,JobRole,JobLevel,DailyRate,HourlyRate,MonthlyRate,MonthlyIncome,PercentSalaryHike,StockOptionLevel,Salary,StandardHours
0,"162,0",51,Male,NaN,NaN,1972,Y,3,NaN,7,NaN,20,NaN,15,15,1,3,3,3.0,3,3.0,No,0.0,6,1,NaN,5,NaN,research director,5,684.0,51.0,6462.0,19537.0,13,0,1000000000,NaN
1,"259,0",52,Male,NaN,NaN,1971,NaN,4,Life Sciences,0,34.0,33,NaN,11,9,3,3,1,3.0,2,3.0,No,NaN,1,1,NaN,5,NaN,manager,5,699.0,65.0,5678.0,19999.0,14,1,1000000000,NaN
2,"319,0",42,Male,Married,NaN,1981,NaN,2,Technical Degree,1,22.0,22,NaN,11,15,3,4,4,NaN,3,3.0,No,0.0,4,1,travel_rarely,3,Research & Development,manager,5,532.0,58.0,4933.0,19232.0,11,0,1000000000,NaN
3,NaN,47,Female,Married,NaN,1976,Y,4,Medical,3,NaN,20,NaN,5,6,1,3,2,NaN,3,3.0,No,NaN,2,0,travel_rarely,2,NaN,research director,4,359.0,82.0,26703.0,17169.0,19,2,1000000000,NaN
4,NaN,46,Female,Divorced,NaN,1977,Y,3,Technical Degree,2,NaN,19,NaN,2,8,1,1,4,3.0,4,3.0,No,0.0,3,0,NaN,5,NaN,sales executive,4,1319.0,45.0,7739.0,NaN,12,1,1000000000,NaN


Revisión de aquellos empleados sin EmployeeID que puedan ser el mismo (coinciden en todas las columnas salvo RemoteWork)

In [9]:
# Saca un DataFrame de los empleados con EmployeeID nulo y que coinciden en todo salvo en RemoteWork

# Filtra las filas con valores nulos en 'EmployeeID'
filas_nulas = df_limpio['EmployeeID'].isnull()
df_filas_nulas = df_limpio[filas_nulas]

# Excluye la columna 'RemoteWork' para buscar duplicados
df_sin_remote = df_filas_nulas.drop(columns=['RemoteWork'])

# Busca duplicados en las filas filtradas
duplicados_sin_RW = df_sin_remote[df_sin_remote.duplicated(keep=False)]

# Extrae los indices
indices_duplicados = duplicados_sin_RW.index

# Crea una lista de los indices con las coincidencias
lista_indices_mismo_empleado = df_limpio.loc[indices_duplicados , :].sort_values(['Age' , 'Gender']).index

Imputación general de nulos para la columna EmployeeID

In [10]:
def asignar_id(valor, contador):
    
    if pd.isnull(valor):
        contador[0] += 1  # Incrementar el contador si el valor es NaN
        id_nulo = f'nan{contador[0]}'  # Devolver el ID con el contador actualizado
        return id_nulo
    
    else:
        return valor.replace(',0' , '')  # Devolver el valor quitándole los decimales

# Reiniciar el contador cada vez que se aplique la función
def aplicar_asignar_id(columna):
    contador = [0]
    return columna.apply(lambda x: asignar_id(x, contador))

# Aplicar la función a la columna 'EmployeeID' y reasignar los resultados
df_limpio['EmployeeID'] = aplicar_asignar_id(df_limpio['EmployeeID'])


Asignación de la misma EmployeeID a los empleados que aparecen en `lista_indices_mismo_empleado`

In [11]:
indice_sublista = 0

for indice in lista_indices_mismo_empleado[1::2]:
    
    dato = lista_indices_mismo_empleado[indice_sublista]
    
    df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]
    
    indice_sublista += 2

/tmp/ipykernel_2297/1808907360.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]


In [12]:
df_limpio.to_csv("../../files/df_to_SQL.csv")

Revisión de nulos

In [13]:
(df_limpio.isnull().sum()/df_limpio.shape[0]) * 100

EmployeeID                    0.000000
Age                           0.000000
Gender                        0.000000
MaritalStatus                40.563020
NumberChildren              100.000000
DateBirth                     0.000000
Over18                       55.854127
Education                     0.000000
EducationField               46.449136
NumCompaniesWorked            0.000000
TotalWorkingYears            32.629559
YearsAtCompany                0.000000
YearsInCurrentRole           97.952655
YearsSinceLastPromotion       0.000000
YearsWithCurrManager          0.000000
EnvironmentSatisfaction       0.000000
JobSatisfaction               0.000000
RelationshipSatisfaction      0.000000
WorkLifeBalance               6.525912
JobInvolvement                0.000000
PerformanceRating            12.092131
Attrition                     0.000000
OverTime                     41.842610
DistanceFromHome              0.000000
RemoteWork                    0.000000
BusinessTravel           

In [14]:
df_limpio.drop (['NumberChildren', 'Over18', 'YearsInCurrentRole', 'Department', 'StandardHours', 'MonthlyIncome'], axis =1, inplace =True)


In [15]:
columnas_desconocido = ['MaritalStatus' , 'EducationField', 'BusinessTravel']

In [16]:
# iteramos por la lista de columnas a las que le vamos a cambiar los nulos por "Uknown"
for columna in columnas_desconocido:
    
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_limpio[columna] = df_limpio[columna].fillna("Unknown")
    
# comprobamos si quedan nulos en las columnas categóricas. 
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_limpio[columnas_desconocido].isnull().sum()

Después del reemplazo usando 'fillna' quedan los siguientes nulos


MaritalStatus     0
EducationField    0
BusinessTravel    0
dtype: int64

In [17]:
numericas = ['WorkLifeBalance', 'PerformanceRating', 'DailyRate', 'HourlyRate', 'TotalWorkingYears', 'OverTime']

In [18]:
df_copy = df_limpio.copy()

In [19]:
# hemos dicho que para las tres columnas restantes aplicaremos los dos métodos para luego compararlos, empezaremos por el IterativeImputer
# instanciamos las clases
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)

# ajustamos y tranformamos los datos
imputer_iterative_imputado = imputer_iterative.fit_transform(df_copy[numericas])

# comprobamos que es lo que nos devuelve, que en este caso es un array también
imputer_iterative_imputado

array([[3.00000000e+00, 3.00000000e+00, 6.84000000e+02, 5.10000000e+01,
        1.11443440e+01, 0.00000000e+00],
       [3.00000000e+00, 3.00000000e+00, 6.99000000e+02, 6.50000000e+01,
        3.40000000e+01, 2.65371642e-01],
       [2.78787368e+00, 3.00000000e+00, 5.32000000e+02, 5.80000000e+01,
        2.20000000e+01, 0.00000000e+00],
       ...,
       [2.00000000e+00, 3.00000000e+00, 8.05000000e+02, 5.70000000e+01,
        1.12565714e+01, 2.79382928e-01],
       [3.00000000e+00, 3.00000000e+00, 9.03000000e+02, 4.10000000e+01,
        9.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 3.00000000e+00, 5.66000000e+02, 7.50000000e+01,
        1.10358300e+01, 2.65843233e-01]])

In [20]:
# hemos dicho que para las tres columnas restantes aplicaremos los dos métodos para luego compararlos, empezaremos por el IterativeImputer
# instanciamos las clases
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)

# ajustamos y tranformamos los datos
imputer_iterative_imputado = imputer_iterative.fit_transform(df_limpio[numericas])

# comprobamos que es lo que nos devuelve, que en este caso es un array también
imputer_iterative_imputado

array([[3.00000000e+00, 3.00000000e+00, 6.84000000e+02, 5.10000000e+01,
        1.11443440e+01, 0.00000000e+00],
       [3.00000000e+00, 3.00000000e+00, 6.99000000e+02, 6.50000000e+01,
        3.40000000e+01, 2.65371642e-01],
       [2.78787368e+00, 3.00000000e+00, 5.32000000e+02, 5.80000000e+01,
        2.20000000e+01, 0.00000000e+00],
       ...,
       [2.00000000e+00, 3.00000000e+00, 8.05000000e+02, 5.70000000e+01,
        1.12565714e+01, 2.79382928e-01],
       [3.00000000e+00, 3.00000000e+00, 9.03000000e+02, 4.10000000e+01,
        9.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 3.00000000e+00, 5.66000000e+02, 7.50000000e+01,
        1.10358300e+01, 2.65843233e-01]])

In [21]:
# en este caso como vamos a aplicar los dos métodos sobre estas columnas vamos a crear dos columnas nuevas
df_limpio[numericas] = imputer_iterative_imputado

# comprobamos los nulos
print(f"Después del 'Iterative' tenemos: \n{df_limpio[numericas].isnull().sum()} nulos")

Después del 'Iterative' tenemos: 
WorkLifeBalance      0
PerformanceRating    0
DailyRate            0
HourlyRate           0
TotalWorkingYears    0
OverTime             0
dtype: int64 nulos


El método **Iterative** parece el más indicado para imputar los nulos, ya que es el que consigue que la diferencia entre la media y la mediana sea más reducida (por ende, menor dispersión de los datos)

In [22]:
df_limpio.to_csv("../../files/df_nulos_imputados.csv")